In [1]:
import sys
import os
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
dataroot = '../ERA5_plus_STD'
filename1 = 'Argo_along_SD.mat'
filename2 = 'Argo_along_SD_Mar.mat'
outname  = 'argo_along_sd.nc'

In [3]:
dat1 = sio.loadmat(os.path.join(dataroot, filename1))
dat2 = sio.loadmat(os.path.join(dataroot, filename2))

In [4]:
dat1

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Oct 13 11:55:25 2021',
 '__version__': '1.0',
 '__globals__': [],
 'Argo_along_SD': array([[(array([[-51.74706   , -58.90377   , -59.491     , -53.25525   ,
                 -53.136     , -58.53671   , -50.698     , -53.18731   ,
                 -53.27403597, -59.73      , -52.285     , -53.2078    ,
                 -60.499     , -53.12722   , -54.164     , -52.96056   ,
                 -50.58650167, -55.533     , -58.35      , -58.639     ,
                 -52.27102   , -52.71153   , -52.42475   , -59.578     ,
                 -60.3       , -52.57976   , -53.2618    , -53.078     ,
                 -53.66      , -54.20901   , -52.969     , -51.54492   ,
                 -54.065     , -52.73430698, -52.759     , -59.835     ,
                 -58.23296333, -51.234     , -59.442     , -53.59607349,
                 -53.673     , -58.813     , -52.371     , -60.365     ,
                 -52.502     , -60.046   

In [5]:
dat2

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Dec  6 23:22:00 2021',
 '__version__': '1.0',
 '__globals__': [],
 'Argo_alongSD_Mar': array([[(array([[737486.80524306],
                [737487.9441088 ],
                [737487.71313657],
                [737487.685     ],
                [737487.52253472],
                [737487.09640046],
                [737488.44150463],
                [737488.26204861],
                [737488.19657407],
                [737488.17791667],
                [737489.99553241],
                [737489.66827546],
                [737489.51357639],
                [737489.50450231],
                [737489.18927083],
                [737490.36491898],
                [737491.27752315],
                [737492.81655093],
                [737493.52071759],
                [737494.99930556],
                [737494.70686343],
                [737495.26456019],
                [737496.79702546],
                [737497.96230324],
 

In [6]:
longname = {
    'lon':    'Longitude',
    'lat':    'Latitude',
    'pre':    'Depth',
    'temp':   'Temperature',
    'sal':    'Salinity',
    }
shortname = {
    'lon':    'lon',
    'lat':    'lat',
    'pre':    'z',
    'temp':   'temp',
    'sal':    'salt',
    }
units = {
    'lon':    'degree_east',
    'lat':    'degree_north',
    'pre':    'm',
    'temp':   'degC',
    'sal':    'psu',
    }

In [7]:
def create_dataset(dat, datname):
    atime2 = dat[datname][0,0]['time2']
    time = xr.DataArray(
        atime2.squeeze(),
        dims = ('time'),
        coords = {'time': atime2.squeeze()},
        attrs = {'long_name': 'time', 'units': 'day of year'}
    )
    apre  = dat[datname][0,0]['pre']
    nlev = apre.shape[0]
    lev = xr.DataArray(
        np.arange(nlev),
        dims = ('lev'),
        coords = {'lev': np.arange(nlev)},
        attrs = {'long_name': 'vertical level', 'units': 'none'}
    )
    out = xr.Dataset()
    for var in ['lon', 'lat']:
        val = dat[datname][0,0][var].squeeze()
        # exclude profiles where z is missing
        val[np.isnan(dat[datname][0,0]['pre'][0,:].squeeze())] = np.nan
        out[var] = xr.DataArray(
            val,
            dims = ('time'),
            coords = {'time': time},
            attrs = {'long_name': longname[var], 'units': units[var]}
        )
    for var in ['pre', 'temp', 'sal']:
        out[shortname[var]] = xr.DataArray(
            dat[datname][0,0][var].transpose(),
            dims = ('time', 'lev'),
            coords = {'time': time, 'lev': lev},
            attrs = {'long_name': longname[var], 'units': units[var]}
        )
    return out

In [8]:
out1 = create_dataset(dat1, 'Argo_along_SD')
out2 = create_dataset(dat2, 'Argo_alongSD_Mar')

In [9]:
out1

<xarray.Dataset>
Dimensions:  (time: 87, lev: 1000)
Coordinates:
  * time     (time) float64 31.9 31.7 31.59 31.1 31.03 ... 55.12 57.0 58.86 58.8
  * lev      (lev) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    lon      (time) float64 -176.5 -107.1 -101.8 -147.4 ... -110.0 -140.6 -145.0
    lat      (time) float64 -51.75 -58.9 -59.49 -53.26 ... -60.87 -52.56 -53.64
    z        (time, lev) float64 -1.12 -2.0 -2.92 ... -1.986e+03 -1.988e+03
    temp     (time, lev) float64 10.91 10.91 10.91 10.91 ... 2.091 2.09 2.089
    salt     (time, lev) float64 34.32 34.32 34.32 34.32 ... 34.72 34.72 34.72

In [10]:
out2

<xarray.Dataset>
Dimensions:  (time: 71, lev: 1000)
Coordinates:
  * time     (time) float64 60.81 61.94 61.71 61.69 ... 86.2 88.55 89.33 89.09
  * lev      (lev) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    lon      (time) float64 -177.7 -175.8 -140.5 -104.4 ... -145.4 -139.7 -125.1
    lat      (time) float64 -52.05 -52.08 -52.22 -59.34 ... -52.73 -53.85 -57.81
    z        (time, lev) float64 -4.52 -6.62 -8.62 ... -1.986e+03 -1.988e+03
    temp     (time, lev) float64 10.5 10.5 10.5 10.5 ... 1.739 1.736 1.732 1.727
    salt     (time, lev) float64 34.45 34.45 34.45 34.45 ... 34.73 34.73 34.73

In [11]:
out = xr.concat([out1, out2], dim='time')

In [12]:
out

<xarray.Dataset>
Dimensions:  (time: 158, lev: 1000)
Coordinates:
  * time     (time) float64 31.9 31.7 31.59 31.1 ... 86.2 88.55 89.33 89.09
  * lev      (lev) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    lon      (time) float64 -176.5 -107.1 -101.8 -147.4 ... -145.4 -139.7 -125.1
    lat      (time) float64 -51.75 -58.9 -59.49 -53.26 ... -52.73 -53.85 -57.81
    z        (time, lev) float64 -1.12 -2.0 -2.92 ... -1.986e+03 -1.988e+03
    temp     (time, lev) float64 10.91 10.91 10.91 10.91 ... 1.736 1.732 1.727
    salt     (time, lev) float64 34.32 34.32 34.32 34.32 ... 34.73 34.73 34.73

In [13]:
out.to_netcdf(os.path.join(dataroot, outname))